# Demo notebook for Three Model Readers
For each model reader, the user is shown how to find the desired variables across model results, how to determine what the correct file input is for the model's reader, and how to run the reader. Note the syntax for each is identical. Any differences in inputs are easily accessible to the user through the methods shown.

In [ ]:
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Choose_Model('')  # Show list of models currently available

In [ ]:
# Find the desired variable
help(MW.Variable_Search)

In [ ]:
# Find what model has the variables...
MW.Variable_Search('temperature')

In [ ]:
# The same output, but only for the requested model
MW.Variable_Search('temperature', model='CTIPe')

In [ ]:
# Find out if the chosen files have the desired variables.
ctipe_file_dir = 'C:/Users/rringuet/Kamodo_Data/CTIPe/Storm_201303/'
MW.Variable_Search('temperature', model='CTIPe', file_dir=ctipe_file_dir)

In [ ]:
# Retrieve model reader and access the model data for the CTIPe model.
# Note one variable chosen depends on pressure level, but the user
# does not need to know that to be able to retrieve and interact with it.
reader = MW.Model_Reader('CTIPe')
kamodo_object_ctipe = reader(ctipe_file_dir, variables_requested=['T_n', 'T_i'])
kamodo_object_ctipe

In [ ]:
# The temperature variables for the requested model
MW.Variable_Search('temperature', model='GITM')

In [ ]:
# Find related variables in the chosen GITM data
gitm_file_dir = 'D:/GITM/Storm_201303/'
MW.Variable_Search('temperature', model='GITM', file_dir=gitm_file_dir)

In [ ]:
# Retrieve model reader and access model data for the GITM model.
# Note the identical syntax!
reader = MW.Model_Reader('GITM')
kamodo_object_gitm = reader(gitm_file_dir, variables_requested=['T_n'])
kamodo_object_gitm

In [ ]:
# repeating for the WACCMX model
MW.Variable_Search('temperature', model='WACCMX')

In [ ]:
# Find related variables in the chosen WACCMX data
# This step also performs any file conversions or pre-processing needed.
waccmx_file_dir = 'D:/WACCMX/storm_201303/'
MW.Variable_Search('temperature', model='WACCMX', file_dir=waccmx_file_dir)

In [ ]:
# Retrieve model reader and access model data for the WACCMX model.
# Note the identical syntax!
reader = MW.Model_Reader('WACCMX')
kamodo_object_waccmx = reader(waccmx_file_dir, variables_requested=['T'])
kamodo_object_waccmx

In [ ]:
# Collect functions into a single Kamodo object.
# Note the names of the coordinates each variable depends on.
from kamodo import Kamodo
kamodo_object = Kamodo()
kamodo_object['CTIPeT_n[K]'] = kamodo_object_ctipe['T_n_ijk']
kamodo_object['GITMT_n[K]'] = kamodo_object_gitm['T_n_ijk']
kamodo_object['WACCMXT[K]'] = kamodo_object_waccmx['T_ijk']
kamodo_object

In [ ]:
# Find out the coordinate ranges for the variables retrieved.
from kamodo import get_defaults
for var in ['CTIPeT_n', 'GITMT_n', 'WACCMXT']:
    print('\n', var)
    defaults_ijk = get_defaults(kamodo_object[var])
    for key in defaults_ijk:
        print(key, len(defaults_ijk[key]), defaults_ijk[key].min(), defaults_ijk[key].max())

In [ ]:
# Print out the datetime object for midnight of the first day of the data.
# Note that the data start on the same day.
print('CTIPe', kamodo_object_ctipe.filedate)
print('GITM', kamodo_object_gitm.filedate)
print('WACCMX', kamodo_object_waccmx.filedate)

In [ ]:
# Plot each. Note that the user can automatically slice a pressure level variable by height!
# Areas of the grid where the height is greater than the pressure level will be blank.
# Change the time and height values as desired.
fig = kamodo_object.plot('CTIPeT_n', plot_partial={'CTIPeT_n': {'time': 12., 'height': 300.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Can slice through any pair of dimensions, as long as the names match the coordinates.
# Note that the maximum and minimum heights of the T_n variable change with time.
# This is because the variable depends on pressure level, which varies.
fig = kamodo_object.plot('CTIPeT_n', plot_partial={'CTIPeT_n': {'lon': 12., 'lat': 30.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
fig = kamodo_object.plot('GITMT_n', plot_partial={'GITMT_n': {'time': 12., 'height': 350.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Example of a default Kamodo plot
kamodo_object.plot('GITMT_n', plot_partial={'GITMT_n': {'time': 12., 'height': 350.}})

In [ ]:
# Directly compare corresponding slices.
# Note the time resolutions are different, but Kamodo automatically interpolates
# to the finer resolution. Also note that the function composition analysis does NOT compensate for
# differences in the data start days. The user must ensure these are the same by retrieving
# any missing data or removing extra data. The datasets can end on different days/times and start 
# at different times BUT must start on the same day.
kamodo_object.plot('CTIPeT_n', 'GITMT_n', 'WACCMXT',
                   plot_partial={'CTIPeT_n': {'time': 12., 'lat': 25., 'height': 350.},
                                 'GITMT_n': {'time': 12., 'lat': 25., 'height': 350.},
                                 'WACCMXT': {'time': 12., 'lat': 25., 'height': 350.}})

In [ ]:
# Demostrate syntax to access the regular interpolator.
# Best for single point calculations.
kamodo_object_ctipe.T_n([12., -20, 20, 350])

In [ ]:
# Demostrate syntax to access the gridded interpolator.
# Best for slice calculations.
kamodo_object_ctipe.T_n_ijk(time=12, lon=-20, lat=20, height=350)

In [ ]:
# At what pressure level does a given height correspond to in the CTIPe data?
# The answer depends on the time, longitude, and latitude.
kamodo_object_ctipe.Plev_ijk(time=12, lon=-20, lat=20, height=350)

In [ ]:
# How does the corresponding pressure level vary at a given latitude and time?
kamodo_object_ctipe.plot('Plev_ijk', plot_partial={'Plev_ijk': {'time': 12., 'lat': 20.,
                                                                'height': 350.}})